# Siamese Network

This notebook shows how to implement siamese network architecture to train a new embedding space.
We will be using Cifar10 dataset, that contains ten differents classes.

#### Dependecies

In [31]:
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch import optim

train_on_gpu = torch.cuda.is_available()

#### Hyperparmeters

In [18]:
# Setting data loaders batch size
batch_size = 64

# percentage of the training data for validation data
valid_size = 0.2

# Specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

#### Data Augmentation Pipeline

In [13]:
# Desing a Data Augmentation pipeline
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

#### Load Dataset

In [14]:
# Load training data
train_data = datasets.CIFAR10('data', train=True, download=True, transform=transform)
# Load testing data
test_data = datasets.CIFAR10('data', train=False, download=True, transform=transform)

Files already downloaded and verified
Files already downloaded and verified


#### Split Validation data

In [15]:
# Shuffling and calculating the split indexes
train_size = len(train_data)
split = int(train_size * (1.0 - valid_size))
shuffle_idx = np.random.permutation(train_size)
train_idx, valid_idx = shuffle_idx[:split], shuffle_idx[split:]

In [104]:
np.random.shuffle(train_idx)

In [110]:
# Defining train and valid samplers
train_sampler = SubsetRandomSampler(train_idx)
np.random.shuffle(train_idx)
train_sampler2 = SubsetRandomSampler(train_idx)

valid_sampler = SubsetRandomSampler(valid_idx)
np.random.shuffle(valid_idx)
valid_sampler2 = SubsetRandomSampler(valid_idx)

#### Data loaders

In [111]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
train_loader2 = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler2)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
valid_loader2 = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

#### Convolutional Network Architecture

In [27]:
class CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # input: 32x32x3 --- output: 32x32x32
        self.conv1_1 = nn.Conv2d(3, 32, 5, padding=2)
        # input: 32x32x32 --- output: 32x32x64
        self.conv1_2 = nn.Conv2d(32, 64, 5, padding=2)
        self.bn1 = nn.BatchNorm2d(64)
        # MaxPooling Here then...
        
        # input: 16x16x32 --- output: 16x16x128
        self.conv2_1 = nn.Conv2d(64, 128, 3, padding=1)
        # input: 16x16x128 --- output: 16x16x256
        self.conv2_2 = nn.Conv2d(126, 256, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(256)
        
        # MaxPooling Here then...
        
        # input: 8x8x256 --- output: 8x8x512
        self.conv3_1 = nn.Conv2d(256, 512, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(512)
        # MaxPooling Here then...
        
        # input: 4x4x512 --- output: 4x4x1024
        self.conv4_1 = nn.Conv2d(512, 1024, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(1024)
        # MaxPooling Here then...
        
        # input: 2x2x1024      
        self.fc1 = nn.Linear(2 * 2 * 1024, 1024)
        self.fc2 = nn.Linear(1024, 256)
        self.fc3 = nn.Linear(256, 64)      
        self.fc4 = nn.Linear(64, 3)        
        
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)


    def forward(self, x):
        
        # Conv Block 1
        x = F.relu(self.conv1_1(x))
        x = F.relu(self.conv1_2(x))
        x = self.pool(self.bn1(x))
        
        # Conv Block 2
        x = F.relu(self.conv2_1(x))
        x = F.relu(self.conv2_2(x))
        x = self.pool(self.bn2(x))
        
        # Conv Block 3
        x = F.relu(self.conv3_1(x))
        x = self.pool(self.bn3(x))
        
        # Conv Block 4
        x = F.relu(self.conv4_1(x))
        x = self.pool(self.bn4(x))
        
        # Flatten
        x = x.view(-1, 2 * 2 * 1024)
        
        # Fully Connected Layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        
        # L2-Normalization
        x_norm2 = torch.sqrt(torch.sum(x**2))
        x = x / x_norm2
        
        return x

In [28]:
model = CNN()
#print(model)

#### Siamese Network

In [30]:
class SiameseNetwork(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.cnn = CNN()
        
    
    def forward(self, x1, x2):
        
        x1 = self.cnn.forward(x1)
        x2 = self.cnn.forward(x2)

        # Calculate Euclidean distance 
        distance = torch.sqrt(torch.sum(x1**2 - x2**2))
        
        return distance
        

In [98]:
model = SiameseNetwork()

#### Training parameters

##### Define Contrastive loss

In [99]:
def contrastive_loss(dist, y):
    margin = 0.5
    
    zeros = torch.zeros_like(dist)
    margin_dist = torch.cat((margin - dist, zeros), dim=0).transpose(0, 1)
    not_same_loss = y * torch.max(margin_dist, 1)
    
    same_loss = -(y - 1) * dist
    
    return -torch.sum(same_loss + not_same_loss)
    

##### Optimizer

In [100]:
optimizer = optim.SGD(model.parameters(), lr = 0.01)

#### Training

In [109]:
n_epochs = 5

for epoch in range(n_epochs):
    train_loss = 0.0
    valid_loss = 0.0
    
    model.train()

    for (x1, y1), (x2, y2) in zip(train_loader, train_loader2):
        y = (y1 != y2)
        
        if train_on_gpu:
            x1, x2, y = x1.cuda(), x2.cuda(), y.cuda()
        
        optimizer.zero_grad()
        
        output = model.forward(x1, x2)
        
        loss = contrastive_loss(output, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
    
    
    model.eval()
    
    for (x1, y1), (x2, y2) in zip(valid_loader, valid_loader2):
        y = (y1 != y2)
        if train_on_gpu:
            x1, x2, y = x1.cuda(), x2.cuda(), y.cuda()
            
        output = model.forward(x1, x2)
        
        loss = contrastive_loss(output, y)
        
        valid_loss += loss.item()
        
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=torch.uint8)
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 

#### Show results